In [74]:
import numpy as np
import csv
import time
import math
import random
from sklearn.model_selection import KFold 
from sklearn.model_selection import train_test_split

In [75]:
def get_dataset(filename):
  with open(filename, newline='') as csvfile:
    reader = csv.DictReader(csvfile)
    ans = []
    for row in reader:
      values = list(map(float, row.values()))
      ans.append(values)
    return np.array(ans)

dataset = get_dataset('sample_data/titanic_data.csv')
dataset
X = dataset[:,1:]
nx = [1, 0, 45, 2, 2, 20]
for i in range(len(X[0])):
  m = np.mean(X[:,i])  
  # print(m)
  X[:,i][X[:,i]<=m] = 0
  X[:,i][X[:,i]>m] = 1  
  nx[i] = 0 if nx[i]<=m else 1
y = dataset[:,0]

In [76]:
nx

[0, 0, 1, 1, 1, 0]

In [77]:
# x = np.array([0,0,0,1,0])
# y = np.array([0,0,0,1,0])

def entropy(x):
  n = len(x)
  zero_count = np.count_nonzero(x)
  one_count = n - zero_count
  p_x_0 = zero_count/n
  p_x_1 = one_count/n
  ans = 0
  ans += p_x_0*math.log(1/p_x_0, 2) if p_x_0>0 else 0
  ans += p_x_1*math.log(1/p_x_1, 2) if p_x_1>0 else 0
  return ans

# entropy(x)

In [78]:
def conditional_entropy(x, y):

  x_y_given_1_arr = x[np.where(y==1)]
  x_1_y_given_1 = np.count_nonzero(x_y_given_1_arr)
  x_0_y_given_1 = len(x_y_given_1_arr) - x_1_y_given_1

  p_y_1 = len(x_y_given_1_arr)/len(y)
  p_x_0_given_y_1 = x_0_y_given_1/len(x_y_given_1_arr)
  p_x_0_and_y_1 = p_x_0_given_y_1*p_y_1

  p_x_1_given_y_1 = x_1_y_given_1/len(x_y_given_1_arr)
  p_x_1_and_y_1 = p_x_1_given_y_1*p_y_1

  x_y_given_0_arr = x[np.where(y==0)]
  x_1_y_given_0 = np.count_nonzero(x_y_given_0_arr)
  x_0_y_given_0 = len(x_y_given_0_arr) - x_1_y_given_0

  p_y_0 = len(x_y_given_0_arr)/len(y)
  p_x_0_given_y_0 = x_0_y_given_0/len(x_y_given_0_arr)
  p_x_0_and_y_0 = p_x_0_given_y_0*p_y_0

  p_x_1_given_y_0 = x_1_y_given_0/len(x_y_given_0_arr)
  p_x_1_and_y_0 = p_x_1_given_y_0*p_y_0

  ans = 0 
  ans += p_x_0_and_y_0 * math.log(1/p_x_0_given_y_0 ,2) if p_x_0_given_y_0 >0 else 0
  ans += p_x_0_and_y_1 * math.log(1/p_x_0_given_y_1 ,2) if p_x_0_given_y_1 >0 else 0
  ans += p_x_1_and_y_0 * math.log(1/p_x_1_given_y_0 ,2) if p_x_1_given_y_0 >0 else 0
  ans += p_x_1_and_y_1 * math.log(1/p_x_1_given_y_1 ,2) if p_x_1_given_y_1 >0 else 0
  return ans

# x = [1]*13
# x.extend([0,1,1])
# x = np.array(x)

# y = [0,0,0,0,0,1,0,0,0,0,0,0,0,1,0,0]
# y = np.array(y)

# conditional_entropy(x,y)

In [79]:
def mutual_information(x, y):  
  return entropy(x) - conditional_entropy(x,y)

# # y = [0,0,0,0,0,1,0,0,0,0,0,0,0,1,0,0]
# y = np.array(y)

# x1 = [1]*13
# x1.extend([0,1,1])
# x1 = np.array(x1)
# print(mutual_information(x1,y))

# x2= [0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0]
# x2 = np.array(x2)
# print(mutual_information(x2,y))

# x3 = [0,1,1,0,0,0,1,1,1,1,0,0,0,1,0,1]
# x3 = np.array(x3)
# print(mutual_information(x3,y))

In [80]:
# X = x1.reshape(16,1)
# X = np.insert(X, 1, x2, axis=1)
# X = np.insert(X, 2, x3, axis=1)
# X


In [ ]:
def split_data(X, y, split_col):
  first_split_X = []
  first_split_y = []
  second_split_X = []
  second_split_y = []
  for i,row in enumerate(X):
    if row[split_col] == 0:
      first_split_X.append(row)
      first_split_y.append(y[i])
    else:
      second_split_X.append(row)
      second_split_y.append(y[i])
  if len(first_split_X):
    first_split_X = np.delete(first_split_X, split_col, axis=1)
  # print(second_split_X)
  if len(second_split_X):
    second_split_X = np.delete(second_split_X, split_col, axis=1)
  return first_split_X, np.array(first_split_y), second_split_X, np.array(second_split_y)

split_data(X, y, split_col)

In [82]:
def best_split(X, y):
  m = len(X[0])
  
  mutual_information_arr = []
  for i in range(m):
    mutual_information_arr.append(mutual_information(X[:,i], y))
  return mutual_information_arr.index(max(mutual_information_arr))
 
split_col = best_split(X, y)

In [83]:
def build_decision_tree(X,y, number_of_samples, orig_columns):

  if len(y) and entropy(y) == 0:
    return "y="+str(y[0])

  if len(X) <= (0.05*number_of_samples):
    y_1 = np.count_nonzero(y)
    return "y=1" if y_1 > (len(y) - y_1) else "y=0"
  
  # when there is only one column left and we delete that column ... X will be array of zero length arrays
  if len(X[0]) == 0:
    y_1 = np.count_nonzero(y)
    return "y=1" if y_1 > (len(y) - y_1) else "y=0"

  split_col = best_split(X, y)
  root = {"Feature considered":orig_columns[split_col]} 
  # root = {"Feature considered":dic[orig_columns[split_col]]} 
  orig_columns.pop(split_col)
  first_split_X, first_split_y, second_split_X, second_split_y = split_data(X, y, split_col)

  if len(first_split_X):
    root["0"] = build_decision_tree(first_split_X, first_split_y, number_of_samples, orig_columns[:])
  else:
    y_1 = np.count_nonzero(second_split_y)
    root["0"] = "y=1" if y_1 > (len(second_split_y) - y_1) else "y=0"

  if len(second_split_X): 
    root["1"] = build_decision_tree(second_split_X, second_split_y, number_of_samples, orig_columns[:])
  else:
    y_1 = np.count_nonzero(first_split_y)
    root["1"] = "y=1" if y_1 > (len(first_split_y) - y_1) else "y=0"

  return root

dic = {0:"Pclass", 1:"Sex", 2:"Age", 3:"Siblings/Spouses Aboard", 4:"Parents/Children Aboard", 5:"Fare"}
root = build_decision_tree(X, y, len(X), [i for i in range(len(X[0]))])

In [23]:
# Fake data
# X = np.array([random.randint(0, 1) for _ in range(16)]).reshape(16,1)
# X = np.insert(X, 1, [random.randint(0, 1) for _ in range(16)], axis=1)
# X = np.insert(X, 2, [random.randint(0, 1) for _ in range(16)], axis=1)
# X = np.insert(X, 3, [random.randint(0, 1) for _ in range(16)], axis=1)
# # print(X)
# y = np.array([random.randint(0, 1) for _ in range(16)])
# y
root = build_decision_tree(X, y, len(X), [i for i in range(len(X[0]))])

In [ ]:
root

In [84]:
def predict(x, root):
  node = root
  while isinstance(node, dict):
    col = node['Feature considered']
    if x[col] == 0:
      node = node['0']
    else:
      node = node['1']
  return float(node.split("=")[-1])

predict(nx, root)

0.0

In [17]:
def accuracy(X, y, root):
  predictions = []
  for x in X:
    predictions.append(predict(x, root))
  predictions = np.array(predictions)
  return (y == predictions).sum() / y.shape[0]

# X = np.array([[1,0,1], [0,0,1], [1,0,0], [0,0,0], [1,1,1]])
# y = np.array([1,0,0,1,1])
# accuracy(X, y, root)

# **My cross validation**

In [86]:
# 4. 7 a
forest = []
number_of_trees = 5
for _ in range(number_of_trees):
    X_train_subset, _, y_train_subset, _ = train_test_split(X, y, test_size=0.2, shuffle=True)
    root = build_decision_tree(X_train_subset, y_train_subset, len(X_train_subset), [i for i in range(len(X_train_subset[0]))])
    forest.append(root)


In [ ]:
forest[4]

In [85]:
# RF - 10-cross validation
# 4. 7 b

i = 0
number_of_splits = 10
l = int(len(X)/number_of_splits)
forests = []
number_of_trees = 5
accs = []

while i+l < len(X):
  X_1, y_1 = X[:i,], y[:i,]
  X_test, y_test = X[i:i+l,], y[i:i+l,]
  X_2, y_2 = X[i+l:,], y[i+l:,]
  X_train = np.concatenate((X_1, X_2)) 
  y_train = np.concatenate((y_1, y_2))  

  acc = 0
  forest = []
  for _ in range(number_of_trees):
    X_train_subset, _, y_train_subset, _ = train_test_split(X_train, y_train, test_size=0.2, shuffle=True)
    root = build_decision_tree(X_train_subset, y_train_subset, len(X_train_subset), [i for i in range(len(X_train_subset[0]))])
    forest.append(root)    
  for root in forest:    
    acc += (accuracy(X_test, y_test, root))
  
  accs.append(acc/(number_of_trees))
  print("Accuracy for "+str(i//l)+"th fold : "+ str(acc/(number_of_trees)))
  
  forests.append(forest)
  i += l

print("10-cross fold average accuracy : "+ str(sum(accs)/(number_of_splits)))

Accuracy for 0th fold : 0.7772727272727272
Accuracy for 1th fold : 0.7999999999999999
Accuracy for 2th fold : 0.7659090909090909
Accuracy for 3th fold : 0.7863636363636364
Accuracy for 4th fold : 0.8272727272727274
Accuracy for 5th fold : 0.7727272727272727
Accuracy for 6th fold : 0.8113636363636363
Accuracy for 7th fold : 0.7863636363636364
Accuracy for 8th fold : 0.8522727272727273
Accuracy for 9th fold : 0.7954545454545454
10-cross fold average accuracy : 0.7975


In [87]:
#4.7 (c)

for root in forest:
  print(predict(nx, root))

0.0
0.0
0.0
0.0
0.0


## Excluding one feature

In [91]:
# 4.8 (a)

random_forest = []
for i in range(len(X[0])):
  X_train_excluding_feature = np.delete(X, i, axis=1)
  root = build_decision_tree(X_train_excluding_feature, y, len(X_train_excluding_feature), [i for i in range(len(X_train_excluding_feature[0]))])
  random_forest.append(root)  


In [96]:
random_forest[3]

{'0': {'0': {'0': {'0': {'0': 'y=0', '1': 'y=0', 'Feature considered': 2},
    '1': {'0': 'y=0', '1': 'y=0', 'Feature considered': 2},
    'Feature considered': 4},
   '1': 'y=0',
   'Feature considered': 3},
  '1': {'0': {'0': {'0': 'y=0', '1': 'y=0', 'Feature considered': 2},
    '1': 'y=1',
    'Feature considered': 4},
   '1': {'0': 'y=0', '1': 'y=0.0', 'Feature considered': 4},
   'Feature considered': 3},
  'Feature considered': 0},
 '1': {'0': {'0': {'0': {'0': 'y=1', '1': 'y=1', 'Feature considered': 2},
    '1': 'y=1',
    'Feature considered': 3},
   '1': {'0': 'y=1', '1': 'y=1.0', 'Feature considered': 2},
   'Feature considered': 4},
  '1': {'0': {'0': {'0': 'y=1', '1': 'y=0', 'Feature considered': 2},
    '1': {'0': 'y=0', '1': 'y=0', 'Feature considered': 2},
    'Feature considered': 3},
   '1': 'y=0.0',
   'Feature considered': 4},
  'Feature considered': 0},
 'Feature considered': 1}

In [89]:
#4.8 (c)

for root in random_forest:
  print(predict(nx, root))

0.0
1.0
0.0
0.0
0.0
0.0


In [54]:
# 4.8 (b)

i = 0
number_of_splits = 10
l = int(len(X)/number_of_splits)
forests = []
number_of_trees = 6
accs = []

while i+l < len(X):
  X_1, y_1 = X[:i,], y[:i,]
  X_test, y_test = X[i:i+l,], y[i:i+l,]
  X_2, y_2 = X[i+l:,], y[i+l:,]
  X_train = np.concatenate((X_1, X_2)) 
  y_train = np.concatenate((y_1, y_2))  
  forest = []
  acc = 0
  for j in range(len(X_train[0])):
    X_train_excluding_feature = np.delete(X_train, j, axis=1)
    X_test_excluding_feature = np.delete(X_test, j, axis=1)
    root = build_decision_tree(X_train_excluding_feature, y_train, len(X_train_excluding_feature), [i for i in range(len(X_train_excluding_feature[0]))])
    forest.append(root)
  for root in forest:    
    acc += (accuracy(X_test, y_test, root))

  accs.append(acc/(number_of_trees))
  print("Accuracy for "+str(i//l)+"th fold : "+ str(acc/(number_of_trees)))
  forests.append(forest)
  i += l

print("10-cross fold average accuracy : "+ str(sum(accs)/(number_of_splits)))

Accuracy for 0th fold : 0.6723484848484849
Accuracy for 1th fold : 0.7348484848484849
Accuracy for 2th fold : 0.6723484848484849
Accuracy for 3th fold : 0.6742424242424242
Accuracy for 4th fold : 0.7007575757575758
Accuracy for 5th fold : 0.6742424242424242
Accuracy for 6th fold : 0.6837121212121212
Accuracy for 7th fold : 0.6704545454545454
Accuracy for 8th fold : 0.725378787878788
Accuracy for 9th fold : 0.6780303030303031
10-cross fold average accuracy : 0.6886363636363636
